In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from surprise import NMF
from surprise import KNNWithMeans
from surprise import accuracy
from surprise.model_selection import KFold
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise.model_selection import cross_validate, train_test_split
from surprise import Reader
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from surprise import KNNBasic
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import os
import json
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data_df = pd.read_csv("../input/hack019.csv")
data_df.head()
data_df['ids'] = data_df['LearnerID'].map(str) + '-' + data_df['MasterYearName'] + '-' + data_df['MasterSubjectName'] 
data_df.head()
reader = Reader(rating_scale=(0, 100))
data = Dataset.load_from_df(data_df[['ids', 'MasterSubjectName', 'Points.1']], reader)
algo = SVD()
# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=15, verbose=True)

In [ ]:
learner_id = 1001

df = data_df.pivot_table(index = ['LearnerID'], values = 'Points.1', columns = 'MasterSubjectName').fillna(0)
df = df.reset_index() 

test= df[df["LearnerID"] == learner_id]
# test = test.drop('LearnerID', axis=1)

model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5, n_jobs=-1)
model_knn.fit(df)
values, indexes = model_knn.kneighbors(test.values.reshape(1,-1))
similar_leaners = dict(zip(indexes[0], values[0]))

sorted_leaners = [(k, similar_leaners[k]) for k in sorted(similar_leaners, key=similar_leaners.get, reverse=True)]

index_arr = []
values_arr = []
for l in sorted_leaners:
    index_arr.append(l[0])
    values_arr.append(l[1])
    
    
similar_learners_df = df.loc[index_arr, :]

similar_learners_df['similarity'] = values_arr
test['similarity'] = 1
similar_learners_df.append(test)
similar_learners_df    


In [ ]:
learner_id = 2058
data = pd.read_csv("../input/hack019.csv")
learner_data = data[data["LearnerID"] == learner_id]

data['year'] = data['MasterYearName'].str.slice(5)
data['year'] = pd.to_numeric(data['year'], errors='coerce').fillna(0).astype(np.int64)

past_data = data[(data['year'] < 3) & (data['year'] != 0) & (data['LearnerID'] != learner_id)]

df = past_data.pivot_table(index = ['LearnerID'], values = 'Points.1', columns = 'MasterSubjectName').fillna(0)
df = df.reset_index() 

learner_pivot = learner_data.pivot_table(index = ['LearnerID'], values = 'Points.1', columns = 'MasterSubjectName').fillna(0)
learner_pivot = learner_pivot.reset_index()

# test= df[df["LearnerID"] == learner_id]
# test = test.drop('LearnerID', axis=1)
final_pivot = pd.concat([learner_pivot, df], ignore_index=False, sort=True).fillna(0)
final_pivot[final_pivot["LearnerID"] == learner_id]


model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=5, n_jobs=-1)
model_knn.fit(df)
values, indexes = model_knn.kneighbors(final_pivot[final_pivot["LearnerID"] == learner_id].values.reshape(1,-1))
similar_leaners = dict(zip(indexes[0], values[0]))

final_learner_pivot = final_pivot[final_pivot["LearnerID"] == learner_id]

sorted_leaners = [(k, similar_leaners[k]) for k in sorted(similar_leaners, key=similar_leaners.get, reverse=True)]

index_arr = []
values_arr = []
for l in sorted_leaners:
    index_arr.append(l[0])
    values_arr.append(l[1])
    
    
similar_learners_df = df.loc[index_arr, :]

similar_learners_df['similarity'] = values_arr
final_learner_pivot['similarity'] = 1
similar_learners_df = pd.concat([similar_learners_df,final_learner_pivot], ignore_index=False, sort=False).fillna(0) 
similar_learners_df 




In [ ]:
data[(data['year'] == 3) & (data['year'].isin(similar_learners_df['LearnerID']))]

In [ ]:
df = df.reset_index() 
df.head()
test.shape

test= df[df["LearnerID"] == 1000]

test = test.drop('LearnerID', axis=1)

In [ ]:
data_df["BPoints"] = data_df["Points.1"] / 10
data_df["BPoints"] = data_df["BPoints"].astype(int)

# bins = [0, 20, 40, 60, , 50, 100]
# labels = [1,2,3,4,5,6]
# df['binned'] = pd.cut(df['percentage'], bins=bins, labels=labels)

In [ ]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(data_df[['ids', 'MasterSubjectName', 'BPoints']], reader)
algo = SVD()
# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
# sim_options = {'name': 'pearson_baseline'}
# algo = KNNBasic(k=40, min_k=1, sim_options=sim_options)
tot_clusters = int(len(df_nmf["uid"].unique()) / 10) + 1
sim_options = {'name': 'pearson_baseline',
               'user_based': False  # compute  similarities between items
               }
algo = KNNBasic(k=tot_clusters, min_k=1, sim_options=sim_options)
trainset, testset = train_test_split(data, test_size=.25)
predictions = algo.fit(trainset).test(testset)
df_nmf = pd.DataFrame(predictions) #predictions     
accuracy.rmse(predictions, verbose=True)

In [ ]:
algo.get_neighbors(5, k=10)

In [ ]:
df_nmf.fillna('')
cluster = []
for index, row in df_nmf.iterrows():
    if "actual_k" in row["details"]:
        cluster.append(row["details"]["actual_k"])
    else:
        cluster.append(0)
df_nmf["cluster"] = cluster

In [ ]:
df_nmf[df_nmf["cluster"] == 1]
df_nmf["cluster"].hist()
plt.hist(df_nmf["cluster"], bins= 50)

In [ ]:
algo = NMF()
# Run 5-fold cross-validation and print results.
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
kf = KFold(n_splits=5)
algo = NMF()

#trainset, testset = train_test_split(data, test_size=.25)
for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

algo = SVD()

#trainset, testset = train_test_split(data, test_size=.25)
for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

algo = SVDpp()

#trainset, testset = train_test_split(data, test_size=.25)
for trainset, testset in kf.split(data):

    # train and test algorithm.
    algo.fit(trainset)
    predictions = algo.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

In [ ]:
algo = NMF()
trainset, testset = train_test_split(data, test_size=.25)
predictions = algo.fit(trainset).test(testset)
df_nmf = pd.DataFrame(predictions) #predictions     
accuracy.rmse(predictions, verbose=True)

algo = SVD()
trainset, testset = train_test_split(data, test_size=.25)
predictions = algo.fit(trainset).test(testset)
df_svd = pd.DataFrame(predictions) #predictions    
accuracy.rmse(predictions, verbose=True)

algo = SVDpp()
trainset, testset = train_test_split(data, test_size=.25)
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)
accuracy.rmse(predictions, verbose=True)


In [ ]:
algo = SVD()
trainset, testset = train_test_split(data, test_size=.25)
predictions = algo.fit(trainset).test(testset)
df_svd = pd.DataFrame(predictions) #predictions    
accuracy.rmse(predictions, verbose=True)

In [ ]:
df_svd.head()

In [ ]:
# toy_story_inner_id = algo.trainset.to_inner_iid(toy_story_raw_id)

# Retrieve inner ids of the nearest neighbors of Toy Story.
toy_story_neighbors = algo.get_neighbors(toy_story_inner_id, k=10)